In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim


from training.train import train_model
from utils.transforms import get_transforms
from utils.evaluation import evaluate_model
from models.ResNet_SC import ModifiedResNet18
from models.ResNet import ResNet18
from dataset.Image_Classification import ImgClassificationDataset, CIFAR10C

In [ ]:
# Hyperparameters
batch_size = 64
num_epochs = 100
lr = 0.0005
num_classes = 10

#loss_fn = "ce"
use_koleo_loss = True

CIFAR10C_ROOT = "./data/cifar/CIFAR-10-C"

# List of all 15 corruption names in CIFAR-10-C (standard ordering)
corruptions = [
	"gaussian_noise", "shot_noise", "impulse_noise",
	"defocus_blur", "glass_blur", "motion_blur", "zoom_blur",
	"snow", "frost", "fog", "brightness",
	"contrast", "elastic_transform", "pixelate",
	"jpeg_compression"
]

# Select device: prefer CUDA, then Apple MPS (for Apple Silicon), otherwise CPU
if torch.cuda.is_available():
	device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
	device = torch.device("mps")
else:
	device = torch.device("cpu")

print(f"Using device: {device}")



Using device: mps


## 1) Load data



In [ ]:
train_transform = get_transforms(split='train')
test_transform = get_transforms(split='test')



train_dataset = datasets.CIFAR10(
	root="./data",          
	train=True,             
	download=True,          
	transform=train_transform
)

test_dataset = datasets.CIFAR10(
	root="./data",
	train=False,
	download=True,
	transform=test_transform
)

train_loader = DataLoader(
	train_dataset,
	batch_size=batch_size,
	shuffle=True,      # shuffle training data each epoch
	num_workers=2      # adjust number of workers to your machine
)

test_loader = DataLoader(
	test_dataset,
	batch_size=batch_size,
	shuffle=False,     # no need to shuffle test/validation data
	num_workers=2
)

## 2) Create model

In [ ]:
# (1) Plain ResNet-18 (no self-correction)
model_plain = ResNet18(
	num_classes=num_classes,
	use_adabn=False,
	use_cbam=False,
	use_proto=False,
	use_rbn=False,
).to(device)


# (2) ResNet-18 + AdaBN
model_adabn = ResNet18(
	num_classes=num_classes,
	use_adabn=True,
	use_cbam=False,
	use_proto=False,
	use_rbn=False,
).to(device)

# (3) ResNet-18 + CBAM
model_cbam = ResNet18(
	num_classes=num_classes,
	use_adabn=False,
	use_cbam=True,
	use_proto=False,
	use_rbn=False,
).to(device)

# (4) ResNet-18 + Prototype Alignment
model_proto = ResNet18(
	num_classes=num_classes,
	use_adabn=False,
	use_cbam=False,
	use_proto=True,
	use_rbn=False,
).to(device)


### 2.1) Model optimizers

In [ ]:

criterion = nn.CrossEntropyLoss()

optim_plain  = optim.SGD(model_plain.parameters(),  lr=lr, momentum=0.9, weight_decay=5e-4)
optim_adabn  = optim.SGD(model_adabn.parameters(),  lr=lr, momentum=0.9, weight_decay=5e-4)
optim_cbam   = optim.SGD(model_cbam.parameters(),   lr=lr, momentum=0.9, weight_decay=5e-4)
optim_proto  = optim.SGD(model_proto.parameters(),  lr=lr, momentum=0.9, weight_decay=5e-4)

# A dictionary to store (model, optimizer) pairs for easy looping:
baseline_dict = {
	"Proto"   : (model_proto, optim_proto),
	"Plain"   : (model_plain, optim_plain),
	"AdaBN"   : (model_adabn, optim_adabn),
	"CBAM"    : (model_cbam,  optim_cbam ),
}

## 3) Train model

In [ ]:



# Track best validation accuracy for each baseline:
best_val_acc = {name: 0.0 for name in baseline_dict.keys()}

for name, (model, optim) in baseline_dict.items():
	train_model(name=name, model=model, optimizer=optim, train_loader=train_loader, test_loader=test_loader, 
				criterion=criterion, device=device, num_epochs=num_epochs)


#train_model(name="ada", model=model_adabn, optimizer=optim_adabn, train_loader=train_loader, test_loader=test_loader,
#			criterion=criterion, device=device, num_epochs=num_epochs)

### Save the models to disk

In [ ]:
'''

# Dictionary mapping a human‐readable name → the actual nn.Module
models_to_save = {
	"plain":  model_plain,
	"adabn":  model_adabn,
	"cbam":   model_cbam,
	"proto":  model_proto,
}

# Ensure the directory exists (in this case, we’re saving to the current working directory)
save_dir = os.getcwd()
print(f"Saving models into: {save_dir}")

for name, model in models_to_save.items():
	# Construct a filename like "model_plain.pth" or "model_adabn.pth"
	filename = f"model_{name}.pth"
	filepath = os.path.join(save_dir, filename)

	# Save only the model’s state dict (weights + buffers)
	torch.save(model.state_dict(), filepath)
	print(f"→ Saved {name} to {filename}")
'''

'\n\n# Dictionary mapping a human‐readable name → the actual nn.Module\nmodels_to_save = {\n\t"plain":  model_plain,\n\t"adabn":  model_adabn,\n\t"cbam":   model_cbam,\n\t"proto":  model_proto,\n}\n\n# Ensure the directory exists (in this case, we’re saving to the current working directory)\nsave_dir = os.getcwd()\nprint(f"Saving models into: {save_dir}")\n\nfor name, model in models_to_save.items():\n\t# Construct a filename like "model_plain.pth" or "model_adabn.pth"\n\tfilename = f"model_{name}.pth"\n\tfilepath = os.path.join(save_dir, filename)\n\n\t# Save only the model’s state dict (weights + buffers)\n\ttorch.save(model.state_dict(), filepath)\n\tprint(f"→ Saved {name} to {filename}")\n'

### Load models from disk

In [ ]:

# Directory to save the models
save_dir = os.getcwd()  # Current working directory


for name, (model, optim) in baseline_dict.items():
	path = os.path.join(save_dir, f"ckpt_epoch_150_{name}.pt")
	print(f"Loading model: from {path}")
	if os.path.exists(path):
		state_dict = torch.load(path, map_location=device)
		model.load_state_dict(state_dict["model_state_dict"])
		print(f"Loaded {name} model from {path}")
	else:
		print(f"File not found: {path}")


Loading model: from /Users/codiri/University/Edinburgh/Lightweight-SCNNs/ckpt_epoch_150_Proto.pt
Loaded Proto model from /Users/codiri/University/Edinburgh/Lightweight-SCNNs/ckpt_epoch_150_Proto.pt
Loading model: from /Users/codiri/University/Edinburgh/Lightweight-SCNNs/ckpt_epoch_150_Plain.pt
Loaded Plain model from /Users/codiri/University/Edinburgh/Lightweight-SCNNs/ckpt_epoch_150_Plain.pt
Loading model: from /Users/codiri/University/Edinburgh/Lightweight-SCNNs/ckpt_epoch_150_AdaBN.pt
Loaded AdaBN model from /Users/codiri/University/Edinburgh/Lightweight-SCNNs/ckpt_epoch_150_AdaBN.pt
Loading model: from /Users/codiri/University/Edinburgh/Lightweight-SCNNs/ckpt_epoch_150_CBAM.pt
Loaded CBAM model from /Users/codiri/University/Edinburgh/Lightweight-SCNNs/ckpt_epoch_150_CBAM.pt


### 3.1) Evaluate model

In [ ]:
# Evaluate total accuracy on val_t_imagenet_loader for all 4 models
accuracies = {}

for name, (model, optim) in baseline_dict.items():
	model.eval()
	correct = 0
	total = 0
	with torch.no_grad():
		for images, labels in test_loader:
			images = images.to(device)
			labels = labels.to(device)
			feats, outputs = model(images)
			_, preds = outputs.max(dim=1)
			correct += (preds == labels).sum().item()
			total += labels.size(0)
	acc = correct / total
	accuracies[name] = acc
	print(f"{name}: Accuracy on Cifar10 val set = {acc*100:.2f}%")

Proto: Accuracy on Cifar10 val set = 15.29%
Plain: Accuracy on Cifar10 val set = 24.47%
AdaBN: Accuracy on Cifar10 val set = 44.82%
CBAM: Accuracy on Cifar10 val set = 24.99%


## 4) Evaluate model on CIFAR-10C dataset

In [ ]:
severity = 1  # Severity level (1-5), where 1 is the least severe
batch_size = 256

# We'll store per-model, per-corruption accuracies here:
results = { name: [] for name in baseline_dict.keys() }

# For each corruption type, build a CIFAR10C loader and measure accuracy:
for corruption in corruptions:
	# Create the CIFAR-10-C dataset for this corruption & severity
	print(f"\nEvaluating corruption: {corruption} (severity={severity})")
	ds_c = CIFAR10C(
		data_dir=CIFAR10C_ROOT,
		corruption=corruption,
		severity=severity,
		transform=test_transform
	)
	loader_c = DataLoader(ds_c,
						  batch_size=batch_size,
						  shuffle=False,
						  num_workers=2,
						  pin_memory=True)

	# For each model, run inference on this loader and compute accuracy
	for name, (model, optim) in baseline_dict.items():
		#print(f"  Evaluating {name}...", end='\n')
		model.eval()  # Set model to evaluation mode
		correct = 0
		total = 0

		with torch.no_grad():
			for images, labels in loader_c:
				images = images.to(device)
				labels = labels.to(device)

				_, outputs = model(images)               # (B, 10)
				_, preds = outputs.max(dim=1)         # (B,)
				correct += (preds == labels).sum().item()
				total += labels.size(0)
				#print(f"  {name}: {correct}/{total} ({100 * correct / total:.2f}%)", end='\r')

		acc = correct / total
		results[name].append(acc)

	print(f"Corruption = {corruption:<15} | "
		  + "  ".join([f"{n}: {results[n][-1]*100:5.2f}%" for n in baseline_dict.keys()]) )

# 5) Summarize average across all corruptions
# ─────────────────────────────────────────────────────────────────────────────

print("\nAverage accuracy over all 15 corruptions (severity=1):")
for name in baseline_dict.keys():
	avg_acc = np.mean(results[name])
	print(f"  {name:<5}  →  {avg_acc*100:5.2f}%")


Evaluating corruption: gaussian_noise (severity=1)


/Users/codiri/installs/miniconda3/envs/SCNN/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Corruption = gaussian_noise  | Proto: 14.83%  Plain: 25.94%  AdaBN: 43.44%  CBAM: 23.15%

Evaluating corruption: shot_noise (severity=1)


### 4.1) Load concatenated CIFAR-10C dataset

In [ ]:
# Load and inspect CIFAR-10-C corruption arrays
print("Loading CIFAR-10-C corruption arrays...")
arr = []


for corruption in corruptions:
	arr_c = np.load(os.path.join(CIFAR10C_ROOT, f"{corruption}.npy"))
	print(f"Corruption: {corruption}, shape: {arr_c.shape}, dtype: {arr_c.dtype}")

	# concatanate all corruption arrays into a single tensor
	arr.append(arr_c)

arr = np.concatenate(arr, axis=0)
print(f"Concatenated array shape: {arr.shape}, dtype: {arr.dtype}")

labels = np.load(os.path.join(CIFAR10C_ROOT, "labels.npy"))
print(f"Labels shape: {labels.shape}, dtype: {labels.dtype}")
all_labels = np.concatenate([labels] * len(corruptions), axis=0)
print(f"All labels shape: {all_labels.shape}, dtype: {all_labels.dtype}")

# Create a daatset from the concatenated array
ds_cifar10c = ImgClassificationDataset(
	data=arr,
	labels=all_labels,
	transform=test_transform
)

# Create a DataLoader for the CIFAR-10-C dataset
loader_cifar10c = DataLoader(
	ds_cifar10c,
	batch_size=batch_size,
	shuffle=False,
	num_workers=2
)

Loading CIFAR-10-C corruption arrays...
Corruption: gaussian_noise, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: shot_noise, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: impulse_noise, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: defocus_blur, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: glass_blur, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: motion_blur, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: zoom_blur, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: snow, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: frost, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: fog, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: brightness, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: contrast, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: elastic_transform, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: pixelate, shape: (50000, 32, 32, 3), dtype: uint8
Corruption: jpeg_compression, shape: (50000, 32, 32, 3), dtype: uint8


### 4.2) Compute ECE over entire CIFAR-10-C dataset

In [ ]:
# compute accuracy and ECE for each model on the CIFAR-10-C dataset
acc_list = []
ece_list = []

for name, (model, optim) in baseline_dict.items():
	print(f"\nEvaluating {name} on CIFAR-10-C dataset...")
	ece, acc = evaluate_model(model=model,
				   loader=loader_cifar10c,
				   device=device)
	acc_list.append(acc)
	ece_list.append(ece)
	print(f"{name} - Accuracy: {acc*100:.2f}%, ECE: {ece:.4f}")

	


Evaluating Proto on CIFAR-10-C dataset...


TypeError: softmax() received an invalid combination of arguments - got (tuple, dim=int), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype = None, *, Tensor out = None)
 * (Tensor input, name dim, *, torch.dtype dtype = None)
